In [1]:
# Import general python packages
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns



In [16]:
policy_set= pd.read_csv("Gelderland_Multi_MORDM_outcomes.csv")

#add column indicating under which scenario which policy was found
policy_set_outcomes= policy_set.iloc[:, -7:]
policy_set_input = policy_set.iloc[:,:51]
policy_set_input

,0_RfR 0,0_RfR 1,0_RfR 2,0_RfR 3,0_RfR 4,1_RfR 0,1_RfR 1,1_RfR 2,1_RfR 3,1_RfR 4,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.4_DikeIncrease 3,A.4_DikeIncrease 4,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.5_DikeIncrease 3,A.5_DikeIncrease 4
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,2,0,0,3
1,0,0,0,0,0,0,0,0,0,0,...,5,0,0,5,0,4,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,7,1,0,2,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,3,7,0,3,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,7,0,0,0,0,4,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,0,0,0,0,0,0,0,0,0,0,...,5,0,0,3,0,6,0,0,0,0
2964,0,0,1,0,0,0,0,0,0,0,...,5,0,0,3,0,6,0,0,0,0
2965,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,3,8,0,0,0,0
2966,0,0,0,0,0,0,0,0,0,0,...,5,0,0,3,0,6,0,0,0,0


In [30]:
from ema_workbench import load_results

results = load_results ('Mutli_MORDM_reference_seed_0.tar.gz')
results

KeyError: "filename 'experiments.csv' not found"

In [22]:
duplicates_RfR = policy_set_input.loc[policy_set_input.iloc[:, 1:25].duplicated()]
duplicates_EWS= policy_set_input.loc[policy_set_input.iloc[:, 26:27].duplicated()]
duplicates_Dikes= policy_set_input.loc[policy_set_input.iloc[:, 26:51].duplicated()]


#669, #1237, #640, #200, #210, #191, #868, #968, #1044

,0_RfR 0,0_RfR 1,0_RfR 2,0_RfR 3,0_RfR 4,1_RfR 0,1_RfR 1,1_RfR 2,1_RfR 3,1_RfR 4,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.4_DikeIncrease 3,A.4_DikeIncrease 4,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.5_DikeIncrease 3,A.5_DikeIncrease 4


In [31]:
df= policy_set_input

# Define a similarity function
def is_close(row1, row2, num_tol=5):
    for col in row1.index:
        if row1[col].dtype == 'float64' or row1[col].dtype == 'int64':
            if abs(row1[col] - row2[col]) > num_tol:
                return False
        else:
            if row1[col] != row2[col]:
                return False
    return True

# Identify close duplicates
def find_close_duplicates(df, num_tol=5):
    close_duplicates = []
    checked = set()
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            if (i, j) not in checked and is_close(df.iloc[i], df.iloc[j], num_tol):
                close_duplicates.append((i, j))
                checked.add((i, j))
                checked.add((j, i))
    return close_duplicates

close_duplicates = find_close_duplicates(df, num_tol=5)

print("Close duplicates (row indices):", close_duplicates)

# Get the rows that are close duplicates
rows_to_keep = set()
for pair in close_duplicates:
    rows_to_keep.update(pair)

close_dup_df = df.loc[rows_to_keep]
close_dup_df

KeyboardInterrupt: 